Approach from Norvig: CSP with depth-first search. 

In [ ]:
def cart_prod(A, B):
  return [a + b for a in A for b in B]

digits = '123456789'
letters = 'ABCDEFGHI'
cols = digits
rows = letters
squares = cart_prod(rows, cols)
unitlist = ([cart_prod(rows, c) for c in cols] + 
            [cart_prod(r, cols) for r in rows] +
            [cart_prod(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')])
units = dict((s, [u for u in unitlist if s in u]) for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in squares)

In [ ]:
def parse_grid(grid):
  values = dict((s, digits) for s in squares)
  for s, d in grid_value(grid).items():
    if d in digits and not assign(values, s, d):
      return False
  return values

def grid_value(grid):
  chars = [c for c in grid if c in digits or c in '0.']
  assert(len(chars)==81)
  return dict(zip(squares, chars))

def assign(values, s, d):
  other_values = values[s].replace(d, '')
  if all(eliminate(values, s, d2) for d2 in other_values):
    return values
  else:
    return False

def eliminate(values, s, d):
  if d not in values[s]:
    return values

  values[s] = values[s].replace(d, '')

  if len(values[s]) == 0:
    return False
  elif len(values[s]) == 1:
    d2 = values[s]
    if not all(eliminate(values, s2, d2) for s2 in peers[s]):
      return False

  for u in units[s]:
    ds = [s for s in u if d in values[s]]
    if len(ds) == 0:
      return False
    elif len(ds) == 1:
      if not assign(values, ds[0], d):
        return False

  return values

In [ ]:
def display(values):
  w = 1 + max(len(values[s]) for s in squares)
  l = '+'.join(['-'*w*3]*3)
  for r in rows:
    print(''.join(values[r+c].center(w)+('|' if c in '36' else '') for c in cols))
    if r in 'CF':
      print(l)
  print()

In [ ]:
def solve(grid):
  return search(parse_grid(grid))

def search(values):
  if values is False:
    return False
  if all(len(values[s])==1 for s in squares):
    return values

  n,s = min((len(values[s]), s) for s in squares if len(values[s])>1)
  return some(search(assign(values.copy(), s, d)) for d in values[s])

def some(seq):
  for e in seq:
    if e: 
      return e
  return False

In [ ]:
grid2 = '6...1...4..5...6....25.98..5.4...3.6.2.....1.13.....82...9.5......7.3....5..2..7.'
display(grid_value(grid2))

6 . . |. 1 . |. . 4 
. . 5 |. . . |6 . . 
. . 2 |5 . 9 |8 . . 
------+------+------
5 . 4 |. . . |3 . 6 
. 2 . |. . . |. 1 . 
1 3 . |. . . |. 8 2 
------+------+------
. . . |9 . 5 |. . . 
. . . |7 . 3 |. . . 
. 5 . |. 2 . |. 7 . 



In [ ]:
display(solve(grid2))

6 9 3 |8 1 2 |7 5 4 
8 1 5 |4 3 7 |6 2 9 
7 4 2 |5 6 9 |8 3 1 
------+------+------
5 8 4 |2 7 1 |3 9 6 
9 2 6 |3 5 8 |4 1 7 
1 3 7 |6 9 4 |5 8 2 
------+------+------
3 7 1 |9 4 5 |2 6 8 
2 6 9 |7 8 3 |1 4 5 
4 5 8 |1 2 6 |9 7 3 

